In [11]:
import pandas as pd
import numpy as np
import os
import hashlib

In [12]:
os.getcwd()

'c:\\Users\\bfran\\Ironhack\\New-York-Payroll-Analysis'

In [13]:
payroll = pd.read_csv('Payroll.csv')

C:\Users\bfran\AppData\Local\Temp\ipykernel_15536\3929464862.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  payroll = pd.read_csv('Payroll.csv')


This below was added/changed. 

In [14]:
string_cols = ['Agency Name', 'Last Name','First Name', 
               'Mid Init', 'Agency Start Date', 'Work Location Borough',
               'Title Description', 'Leave Status as of June 30','Pay Basis']

# Strip extra spaces and uppercase all string columns
for col in string_cols:
    payroll[col] = payroll[col].str.strip().str.upper()

This was added.

In [15]:
# Because other options for filling nulls in the borough column proved too memory intensive, filling them with "WITHHELD" is the best option. 
payroll['Work Location Borough'].fillna('WITHHELD',inplace=True)

# Create a list of appropriate borough options
boroughs = ['BROOKLYN', 'MANHATTAN', 'BRONX', 'RICHMOND', 'QUEENS','WESTCHESTER', 'NASSAU', 'ORANGE','PUTNAM', 'WITHHELD']

# Replace all non-WITHHELD locations that are not NYC boroughs with OTHER
payroll['Work Location Borough'] = np.where(payroll['Work Location Borough'].isin(boroughs),
                                            payroll['Work Location Borough'],
                                            'OTHER')

In [16]:
payroll.columns

Index(['Fiscal Year', 'Payroll Number', 'Agency Name', 'Last Name',
       'First Name', 'Mid Init', 'Agency Start Date', 'Work Location Borough',
       'Title Description', 'Leave Status as of June 30', 'Base Salary',
       'Pay Basis', 'Regular Hours', 'Regular Gross Paid', 'OT Hours',
       'Total OT Paid', 'Total Other Pay'],
      dtype='object')

In [8]:
payroll['Agency Start Date'] = pd.to_datetime(payroll['Agency Start Date'], errors='coerce')
payroll['Agency Start Date'].dropna(inplace=True)

,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Agency Start Date,agency_start_date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
160818,2020,742.0,DEPT OF ED PEDAGOGICAL,GOLDBERG,ADAM,S,12/31/9999,NaT,MANHATTAN,TEACHER SPECIAL EDUCATION,ACTIVE,124909.00,PER ANNUM,0.0,122242.80,0.0,0.0,36229.15
168070,2020,742.0,DEPT OF ED PEDAGOGICAL,CHIARELLO,JAMES,L,12/31/9999,NaT,MANHATTAN,TEACHER,ACTIVE,124909.00,PER ANNUM,0.0,122242.80,0.0,0.0,13012.34
168303,2020,742.0,DEPT OF ED PEDAGOGICAL,ROSWELL,CAROLINE,D,12/31/9999,NaT,MANHATTAN,TEACHER,ACTIVE,124909.00,PER ANNUM,0.0,122242.80,0.0,0.0,12876.50
168523,2020,742.0,DEPT OF ED PEDAGOGICAL,ROTCHFORD,DOROTHY,R,12/31/9999,NaT,MANHATTAN,TEACHER,ACTIVE,124909.00,PER ANNUM,0.0,122242.80,0.0,0.0,12759.82
168947,2020,742.0,DEPT OF ED PEDAGOGICAL,DAQUI,CAROL,NaN,12/31/9999,NaT,MANHATTAN,TEACHER,ACTIVE,124909.00,PER ANNUM,0.0,122242.80,0.0,0.0,12536.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5448010,2023,744.0,DEPT OF ED PARA PROFESSIONALS,WILLIAMS,CAROLYN,NaN,12/31/9999,NaT,MANHATTAN,ANNUAL ED PARA,ACTIVE,47723.00,PER ANNUM,0.0,47723.04,0.0,0.0,6628.45
5448909,2023,744.0,DEPT OF ED PARA PROFESSIONALS,ZIELENSKI,KATHLEEN,J,12/31/9999,NaT,MANHATTAN,SUBSTITUTE ED PARA,ACTIVE,166.67,PER DAY,0.0,5770.44,0.0,0.0,166.67
5455886,2023,745.0,DEPT OF ED HRLY SUPPORT STAFF,DOMINIQUE,MARIE,B,12/31/9999,NaT,MANHATTAN,F/T SCHOOL AIDE,ACTIVE,17.04,PER HOUR,0.0,22893.30,0.0,0.0,12834.89
5469018,2023,745.0,DEPT OF ED HRLY SUPPORT STAFF,REYES,HECTOR,L,12/31/9999,NaT,MANHATTAN,F/T SCHOOL AIDE,ACTIVE,17.04,PER HOUR,0.0,18080.23,0.0,0.0,8465.66


In [6]:
# Identifying information (names) for some job titles have been removed for the safety and protection of the individuals filling those job posts. 
# Because I want to generate IDs for each person, however, I need to have something in those columns. X will suffice. 
payroll['First Name'].fillna('X', inplace=True)
payroll['Last Name'].fillna('X', inplace=True)

In [7]:
# Because Title Description and Agency Start Date will form part of the Worker ID, I need something in those columns as well. 
# However, since only 63 start dates and only 96 title descriptions are missing out of a dataset of 5,000,000 rows, it's okay to drop them. 
payroll.dropna(subset=['Agency Start Date','Title Description'], inplace=True)

In [1]:
payroll.columns.dtype

NameError: name 'payroll' is not defined

In [10]:
# Function to generate worker ID 
def generate_custom_worker_id(row, id_dict, count_dict):
    # Take the first letter of the first and last names
    first_initial = row['First Name'][0]
    last_initial = row['Last Name'][0]
    
    # Take the first letters of each word in the job title
    job_title_initials = ''.join([word[0] for word in row['Title Description'].split()])
    
    # Use the start date
    start_date = row['Agency Start Date'].replace('/', '').replace('-','')
    
    # Create a unique key based on the relevant fields
    unique_key = (first_initial, last_initial, job_title_initials, start_date)
    
    # Base ID without any counter
    base_id = f"{first_initial}{last_initial}{job_title_initials}_{start_date}"
    
    # Check if the unique key is already in the dictionary
    if unique_key not in id_dict:
        if base_id not in count_dict:
            count_dict[base_id] = 1
        else:
            count_dict[base_id] += 1
        
        worker_id = f"{base_id}_{count_dict[base_id]}" if count_dict[base_id] > 1 else base_id
        id_dict[unique_key] = worker_id
    else:
        worker_id = id_dict[unique_key]
    
    return worker_id

# Dictionaries to keep track of generated IDs and counts for each unique combination
id_dict = {}
count_dict = {}

# Apply the function to each row to create the 'Worker ID' column
payroll['Worker ID'] = payroll.apply(lambda row: generate_custom_worker_id(row, id_dict, count_dict), axis=1)

payroll

,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay,Worker ID
0,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,BEREZIN,MIKHAIL,NaN,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,MBEPM_08102015
1,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,GEAGER,VERONICA,M,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,VGEPM_09122016
2,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,RAMANI,SHRADDHA,NaN,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,SREPM_02222016
3,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,ROTTA,JONATHAN,D,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,JREPM_09162013
4,2020,17.0,OFFICE OF EMERGENCY MANAGEMENT,WILSON II,ROBERT,P,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,RWEPM_04302018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,JUAN,NaN,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01,JZP_10022017
5662709,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,LUIS,NaN,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96,LZP_08212017
5662710,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA,PAUL,E,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28,PZSP_10072019
5662711,2023,996.0,NYC HOUSING AUTHORITY,ZUNIGA BERMEO,HUGO,E,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00,HZP_07062022


In [ ]:
payroll['Fiscal Year'].value_counts()

Fiscal Year
2022    613003
2019    592405
2020    590141
2015    577874
2021    573473
2017    562259
2023    552935
2018    546138
2016    544810
2014    509516
Name: count, dtype: int64

In [11]:
# Drops Payroll Number because it is just a numerical category of Agency Name and has missings where Agency Name does not. 
# Drops identity information because it is irrelevant and unethical to use. 
drop_columns = ['Payroll Number','Last Name','First Name','Mid Init']
payroll.drop(columns = drop_columns, inplace=True)

In [ ]:
payroll.columns

Index(['Fiscal Year', 'Agency Name', 'Agency Start Date',
       'Work Location Borough', 'Title Description',
       'Leave Status as of June 30', 'Base Salary', 'Pay Basis',
       'Regular Hours', 'Regular Gross Paid', 'OT Hours', 'Total OT Paid',
       'Total Other Pay', 'Worker ID'],
      dtype='object')

In [12]:
column_order = ['Worker ID','Fiscal Year', 'Agency Name', 'Agency Start Date',
       'Work Location Borough', 'Title Description',
       'Leave Status as of June 30', 'Base Salary', 'Pay Basis',
       'Regular Hours', 'Regular Gross Paid', 'OT Hours', 'Total OT Paid',
       'Total Other Pay']
payroll = payroll[column_order]
payroll

,Worker ID,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,MBEPM_08102015,2020,OFFICE OF EMERGENCY MANAGEMENT,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
1,VGEPM_09122016,2020,OFFICE OF EMERGENCY MANAGEMENT,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
2,SREPM_02222016,2020,OFFICE OF EMERGENCY MANAGEMENT,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
3,JREPM_09162013,2020,OFFICE OF EMERGENCY MANAGEMENT,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
4,RWEPM_04302018,2020,OFFICE OF EMERGENCY MANAGEMENT,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662708,JZP_10022017,2023,NYC HOUSING AUTHORITY,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01
5662709,LZP_08212017,2023,NYC HOUSING AUTHORITY,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96
5662710,PZSP_10072019,2023,NYC HOUSING AUTHORITY,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28
5662711,HZP_07062022,2023,NYC HOUSING AUTHORITY,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00


In [13]:
# Reset Index
payroll.reset_index(drop=True, inplace=True)

# Add a true index column
payroll.reset_index(drop=False, inplace=True)

# Rename the new index column if needed
payroll.rename(columns={'index': 'True Index'}, inplace=True)

C:\Users\bfran\AppData\Local\Temp\ipykernel_12928\4091747183.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payroll.rename(columns={'index': 'True Index'}, inplace=True)


In [14]:
payroll

,True Index,Worker ID,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay
0,0,MBEPM_08102015,2020,OFFICE OF EMERGENCY MANAGEMENT,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
1,1,VGEPM_09122016,2020,OFFICE OF EMERGENCY MANAGEMENT,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
2,2,SREPM_02222016,2020,OFFICE OF EMERGENCY MANAGEMENT,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
3,3,JREPM_09162013,2020,OFFICE OF EMERGENCY MANAGEMENT,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
4,4,RWEPM_04302018,2020,OFFICE OF EMERGENCY MANAGEMENT,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662549,5662549,JZP_10022017,2023,NYC HOUSING AUTHORITY,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01
5662550,5662550,LZP_08212017,2023,NYC HOUSING AUTHORITY,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96
5662551,5662551,PZSP_10072019,2023,NYC HOUSING AUTHORITY,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28
5662552,5662552,HZP_07062022,2023,NYC HOUSING AUTHORITY,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00


In [15]:
payroll.isnull().sum()

True Index                    0
Worker ID                     0
Fiscal Year                   0
Agency Name                   0
Agency Start Date             0
Work Location Borough         0
Title Description             0
Leave Status as of June 30    0
Base Salary                   0
Pay Basis                     0
Regular Hours                 0
Regular Gross Paid            0
OT Hours                      0
Total OT Paid                 0
Total Other Pay               0
dtype: int64

In [16]:
# Define a function to convert pay based on pay basis
def convert_pay(row):
    if row['Pay Basis'] == 'per Day':
        return row['Base Salary'] * 365
    elif row['Pay Basis'] == 'per Hour':
        return row['Base Salary'] * 40 * 48
    else:
        return row['Base Salary']

# Apply the function to each row to create the 'Converted Pay' column
payroll['Converted Salary'] = payroll.apply(convert_pay, axis=1)
payroll

C:\Users\bfran\AppData\Local\Temp\ipykernel_12928\3758434699.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payroll['Converted Salary'] = payroll.apply(convert_pay, axis=1)


,True Index,Worker ID,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay,Converted Salary
0,0,MBEPM_08102015,2020,OFFICE OF EMERGENCY MANAGEMENT,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00
1,1,VGEPM_09122016,2020,OFFICE OF EMERGENCY MANAGEMENT,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00
2,2,SREPM_02222016,2020,OFFICE OF EMERGENCY MANAGEMENT,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00
3,3,JREPM_09162013,2020,OFFICE OF EMERGENCY MANAGEMENT,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00
4,4,RWEPM_04302018,2020,OFFICE OF EMERGENCY MANAGEMENT,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662549,5662549,JZP_10022017,2023,NYC HOUSING AUTHORITY,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01,315.07
5662550,5662550,LZP_08212017,2023,NYC HOUSING AUTHORITY,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96,315.07
5662551,5662551,PZSP_10072019,2023,NYC HOUSING AUTHORITY,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28,360.08
5662552,5662552,HZP_07062022,2023,NYC HOUSING AUTHORITY,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00,315.07


In [17]:
# Calculate the sum of the specified columns and create the 'Total Pay' column
payroll['Total Pay'] = payroll[['Regular Gross Paid', 'Total OT Paid', 'Total Other Pay']].sum(axis=1)
payroll

C:\Users\bfran\AppData\Local\Temp\ipykernel_12928\1727912441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payroll['Total Pay'] = payroll[['Regular Gross Paid', 'Total OT Paid', 'Total Other Pay']].sum(axis=1)


,True Index,Worker ID,Fiscal Year,Agency Name,Agency Start Date,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay,Converted Salary,Total Pay
0,0,MBEPM_08102015,2020,OFFICE OF EMERGENCY MANAGEMENT,08/10/2015,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21
1,1,VGEPM_09122016,2020,OFFICE OF EMERGENCY MANAGEMENT,09/12/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21
2,2,SREPM_02222016,2020,OFFICE OF EMERGENCY MANAGEMENT,02/22/2016,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21
3,3,JREPM_09162013,2020,OFFICE OF EMERGENCY MANAGEMENT,09/16/2013,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21
4,4,RWEPM_04302018,2020,OFFICE OF EMERGENCY MANAGEMENT,04/30/2018,BROOKLYN,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.00,PER ANNUM,1820.0,84698.21,0.00,0.00,0.00,86005.00,84698.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662549,5662549,JZP_10022017,2023,NYC HOUSING AUTHORITY,10/02/2017,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1149.00,77575.47,3752.01,315.07,163245.68
5662550,5662550,LZP_08212017,2023,NYC HOUSING AUTHORITY,08/21/2017,BRONX,PAINTER,ACTIVE,315.07,PER DAY,1820.0,81918.20,1037.75,70063.81,3206.96,315.07,155188.97
5662551,5662551,PZSP_10072019,2023,NYC HOUSING AUTHORITY,10/07/2019,QUEENS,SUPERVISOR PAINTER,ACTIVE,360.08,PER DAY,1820.0,93620.80,908.00,70035.56,617.28,360.08,164273.64
5662552,5662552,HZP_07062022,2023,NYC HOUSING AUTHORITY,07/06/2022,QUEENS,PAINTER,ACTIVE,315.07,PER DAY,1771.0,79082.62,1005.75,67869.89,0.00,315.07,146952.51


In [18]:
payroll.columns.str.strip().str.lower().str.replace(' ','_')

Index(['true_index', 'worker_id', 'fiscal_year', 'agency_name',
       'agency_start_date', 'work_location_borough', 'title_description',
       'leave_status_as_of_june_30', 'base_salary', 'pay_basis',
       'regular_hours', 'regular_gross_paid', 'ot_hours', 'total_ot_paid',
       'total_other_pay', 'converted_salary', 'total_pay'],
      dtype='object')

In [1]:
# Import Necessary Packages
import pandas as pd
import numpy as np
import hashlib

# Create function that cleans dataset in new data analysis notebook.
def get_cleaned_df():
    
    # Load Data
    payroll = pd.read_csv('Payroll.csv')

    # Define string columns to target for cleaning
    string_cols = ['Agency Name', 'Last Name','First Name', 
               'Mid Init', 'Agency Start Date', 'Work Location Borough',
               'Title Description', 'Leave Status as of June 30','Pay Basis']

    # Strip extra spaces and uppercase all string columns
    for col in string_cols:
        payroll[col] = payroll[col].str.strip().str.upper()

    # Because other options for filling nulls in the borough column proved too memory intensive, filling them with "WITHHELD" is the best option. 
    payroll['Work Location Borough'].fillna('WITHHELD',inplace=True)

    # Create a list of appropriate borough options
    boroughs = ['BROOKLYN', 'MANHATTAN', 'BRONX', 'RICHMOND', 'QUEENS','WESTCHESTER', 'NASSAU', 'ORANGE','PUTNAM', 'WITHHELD']

    # Replace all non-WITHHELD locations that are not NYC boroughs with OTHER
    payroll['Work Location Borough'] = np.where(payroll['Work Location Borough'].isin(boroughs),
                                                payroll['Work Location Borough'],
                                                'OTHER')
    
    # Convert Agency Start Date to pandas date time for easy use in SQL
    # Drop Errors and nulls
    payroll['Agency Start Date'] = pd.to_datetime(payroll['Agency Start Date'], errors='coerce')
    payroll['Agency Start Date'].dropna(inplace=True)
    
    # Identifying information (names) for some job titles have been removed for the safety and protection of the individuals filling those job posts. 
    # Because I want to generate IDs for each person, however, I need to have something in those columns. X will suffice. 
    payroll['First Name'].fillna('X', inplace=True)
    payroll['Last Name'].fillna('X', inplace=True)

    # Because Title Description and Agency Start Date will form part of the Worker ID, I need something in those columns as well. 
    # However, since only 63 start dates and only 96 title descriptions are missing out of a dataset of 5,000,000 rows, it's okay to drop them. 
    payroll.dropna(subset=['Agency Start Date','Title Description'], inplace=True)

    # Function to generate worker ID 
    def generate_custom_worker_id(row, id_dict, count_dict):
        # Take the first letter of the first and last names
        first_initial = row['First Name'][0]
        last_initial = row['Last Name'][0]
    
        # Take the first letters of each word in the job title
        job_title_initials = ''.join([word[0] for word in row['Title Description'].split()])
    
        # Use the start date
        start_date = row['Agency Start Date'].replace('/', '').replace('-','')
    
        # Create a unique key based on the relevant fields
        unique_key = (first_initial, last_initial, job_title_initials, start_date)
    
        # Base ID without any counter
        base_id = f"{first_initial}{last_initial}{job_title_initials}_{start_date}"
    
        # Check if the unique key is already in the dictionary
        if unique_key not in id_dict:
            if base_id not in count_dict:
                count_dict[base_id] = 1
            else:
                count_dict[base_id] += 1
        
            worker_id = f"{base_id}_{count_dict[base_id]}" if count_dict[base_id] > 1 else base_id
            id_dict[unique_key] = worker_id
        else:
            worker_id = id_dict[unique_key]
    
        return worker_id

    # Dictionaries to keep track of generated IDs and counts for each unique combination
    id_dict = {}
    count_dict = {}

    # Apply the function to each row to create the 'Worker ID' column
    payroll['Worker ID'] = payroll.apply(lambda row: generate_custom_worker_id(row, id_dict, count_dict), axis=1)

    # Drops Payroll Number because it is just a numerical category of Agency Name and has missings where Agency Name does not. 
    # Drops identity information because it is irrelevant and unethical to use. 
    drop_columns = ['Payroll Number','Last Name','First Name','Mid Init']
    payroll.drop(columns = drop_columns, inplace=True)

    # Set new column order
    column_order = ['Worker ID','Fiscal Year', 'Agency Name', 'Agency Start Date',
       'Work Location Borough', 'Title Description',
       'Leave Status as of June 30', 'Base Salary', 'Pay Basis',
       'Regular Hours', 'Regular Gross Paid', 'OT Hours', 'Total OT Paid',
       'Total Other Pay']
    
    # Apply new column order
    payroll = payroll[column_order]

    # Reset Index
    payroll.reset_index(drop=True, inplace=True)

    # Add a true index column
    payroll.reset_index(drop=False, inplace=True)

    # Rename the new index column if needed
    payroll.rename(columns={'index': 'True Index'}, inplace=True)

    # Define a function to convert pay based on pay basis
    def convert_pay(row):
        if row['Pay Basis'] == 'per Day':
            return row['Base Salary'] * 365
        elif row['Pay Basis'] == 'per Hour':
            return row['Base Salary'] * 40 * 48
        else:
            return row['Base Salary']

    # Apply the function to each row to create the 'Converted Pay' column
    payroll['Converted Salary'] = payroll.apply(convert_pay, axis=1)

    # Calculate the sum of the specified columns and create the 'Total Pay' column
    payroll['Total Pay'] = payroll[['Regular Gross Paid', 'Total OT Paid', 'Total Other Pay']].sum(axis=1)

    # Convert column names to be more python and sql friendly
    payroll.columns = payroll.columns.str.strip().str.lower().str.replace(' ','_')

    # Set new column order
    columns_order = ['worker_id', 'fiscal_year', 'agency_name',
       'agency_start_date', 'work_location_borough', 'title_description',
       'leave_status_as_of_june_30', 'base_salary', 'pay_basis',
       'regular_hours', 'regular_gross_paid', 'ot_hours', 'total_ot_paid',
       'total_other_pay', 'converted_salary', 'total_pay','true_index']
    
    # apply new column order
    payroll = payroll[columns_order]
    
    return payroll

In [2]:
# Test to make sure it works. It does. 
# payroll = get_cleaned_df()

C:\Users\bfran\AppData\Local\Temp\ipykernel_6024\3297766089.py:10: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  payroll = pd.read_csv('Payroll.csv')
